![JohnSnowLabs](https://nlp.johnsnowlabs.com/assets/images/logo.png)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/JohnSnowLabs/spark-nlp-workshop/blob/master/tutorials/Certification_Trainings/Healthcare/Spark%20v2.7.6%20Notebooks/24.Improved_Entity_Resolvers_in_SparkNLP_with_sBert.ipynb)

# 24. Improved Entity Resolvers in Spark NLP with sBert

In [ ]:
import json

from google.colab import files

license_keys = files.upload()

with open(list(license_keys.keys())[0]) as f:
    license_keys = json.load(f)

In [ ]:
%%capture
for k,v in license_keys.items(): 
    %set_env $k=$v

!wget https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp-workshop/master/jsl_colab_setup.sh
!bash jsl_colab_setup.sh -p 2.4.4

! pip install spark-nlp-display

In [ ]:
import json
import sys, os, time
from pyspark.ml import Pipeline,PipelineModel
from pyspark.sql import SparkSession

from sparknlp.annotator import *
from sparknlp_jsl.annotator import *
from sparknlp.base import *
from sparknlp.util import *
import sparknlp_jsl
import sparknlp
from pyspark.sql import functions as F

params = {"spark.driver.memory":"16G",
"spark.kryoserializer.buffer.max":"2000M",
"spark.driver.maxResultSize":"2000M"}

spark = sparknlp_jsl.start(license_keys['SECRET'],params=params)

print (sparknlp.version())
print (sparknlp_jsl.version())

2.7.4
2.7.6


In [ ]:
spark

## Snomed pipeline

In [ ]:
documentAssembler = DocumentAssembler()\
      .setInputCol("text")\
      .setOutputCol("ner_chunk")

sbert_embedder = BertSentenceEmbeddings\
      .pretrained('sbiobert_base_cased_mli', 'en','clinical/models')\
      .setInputCols(["ner_chunk"])\
      .setOutputCol("sbert_embeddings")
    
snomed_ct_resolver = SentenceEntityResolverModel.pretrained("sbiobertresolve_snomed_findings","en", "clinical/models") \
      .setInputCols(["ner_chunk", "sbert_embeddings"]) \
      .setOutputCol("snomed_code")\
      .setDistanceFunction("EUCLIDEAN")

snomed_pipelineModel = PipelineModel(
    stages = [
        documentAssembler,
        sbert_embedder,
        snomed_ct_resolver])

snomed_lp = LightPipeline(snomed_pipelineModel)


sbiobert_base_cased_mli download started this may take some time.
Approximate size to download 384.3 MB
[OK!]
sbiobertresolve_snomed_findings download started this may take some time.
Approximate size to download 1.4 GB
[OK!]


## ICD10CM pipeline

In [ ]:
documentAssembler = DocumentAssembler()\
      .setInputCol("text")\
      .setOutputCol("ner_chunk")

sbert_embedder = BertSentenceEmbeddings\
      .pretrained('sbiobert_base_cased_mli', 'en','clinical/models')\
      .setInputCols(["ner_chunk"])\
      .setOutputCol("sbert_embeddings")
    
icd10_resolver = SentenceEntityResolverModel.pretrained("sbiobertresolve_icd10cm_augmented","en", "clinical/models") \
      .setInputCols(["ner_chunk", "sbert_embeddings"]) \
      .setOutputCol("icd10cm_code")\
      .setDistanceFunction("EUCLIDEAN")

icd_pipelineModel = PipelineModel(
    stages = [
        documentAssembler,
        sbert_embedder,
        icd10_resolver])

icd_lp = LightPipeline(icd_pipelineModel)


sbiobert_base_cased_mli download started this may take some time.
Approximate size to download 384.3 MB
[OK!]
sbiobertresolve_icd10cm_augmented download started this may take some time.
Approximate size to download 1.2 GB
[OK!]


## RxNorm pipeline

In [ ]:
documentAssembler = DocumentAssembler()\
      .setInputCol("text")\
      .setOutputCol("ner_chunk")

sbert_embedder = BertSentenceEmbeddings\
      .pretrained('sbiobert_base_cased_mli', 'en','clinical/models')\
      .setInputCols(["ner_chunk"])\
      .setOutputCol("sbert_embeddings")
    
rxnorm_resolver = SentenceEntityResolverModel.pretrained("sbiobertresolve_rxnorm","en", "clinical/models") \
      .setInputCols(["ner_chunk", "sbert_embeddings"]) \
      .setOutputCol("rxnorm_code")\
      .setDistanceFunction("EUCLIDEAN")

rxnorm_pipelineModel = PipelineModel(
    stages = [
        documentAssembler,
        sbert_embedder,
        rxnorm_resolver])

rxnorm_lp = LightPipeline(rxnorm_pipelineModel)


sbiobert_base_cased_mli download started this may take some time.
Approximate size to download 384.3 MB
[OK!]
sbiobertresolve_rxnorm download started this may take some time.
Approximate size to download 810.7 MB
[OK!]


## CPT pipeline

In [ ]:
documentAssembler = DocumentAssembler()\
      .setInputCol("text")\
      .setOutputCol("ner_chunk")

sbert_embedder = BertSentenceEmbeddings\
      .pretrained('sbiobert_base_cased_mli', 'en','clinical/models')\
      .setInputCols(["ner_chunk"])\
      .setOutputCol("sbert_embeddings")
    
cpt_resolver = SentenceEntityResolverModel.pretrained("sbiobertresolve_cpt_procedures_augmented","en", "clinical/models") \
      .setInputCols(["ner_chunk", "sbert_embeddings"]) \
      .setOutputCol("cpt_code")\
      .setDistanceFunction("EUCLIDEAN")

cpt_pipelineModel = PipelineModel(
    stages = [
        documentAssembler,
        sbert_embedder,
        cpt_resolver])

cpt_lp = LightPipeline(cpt_pipelineModel)


sbiobert_base_cased_mli download started this may take some time.
Approximate size to download 384.3 MB
[OK!]
sbiobertresolve_cpt_procedures_augmented download started this may take some time.
Approximate size to download 112.2 MB
[OK!]


## ICD10CM-HCC pipeline

In [ ]:
documentAssembler = DocumentAssembler()\
      .setInputCol("text")\
      .setOutputCol("ner_chunk")

sbert_embedder = BertSentenceEmbeddings\
      .pretrained('sbiobert_base_cased_mli', 'en','clinical/models')\
      .setInputCols(["ner_chunk"])\
      .setOutputCol("sbert_embeddings")
    
hcc_resolver = SentenceEntityResolverModel.pretrained("sbiobertresolve_icd10cm_augmented_billable_hcc","en", "clinical/models") \
      .setInputCols(["ner_chunk", "sbert_embeddings"]) \
      .setOutputCol("icd10cm_hcc_code")\
      .setDistanceFunction("EUCLIDEAN")

hcc_pipelineModel = PipelineModel(
    stages = [
        documentAssembler,
        sbert_embedder,
        hcc_resolver])

hcc_lp = LightPipeline(hcc_pipelineModel)


sbiobert_base_cased_mli download started this may take some time.
Approximate size to download 384.3 MB
[OK!]
sbiobertresolve_icd10cm_augmented_billable_hcc download started this may take some time.
Approximate size to download 1.4 GB
[OK!]


## All the resolvers in the same pipeline 
### (just to show how it is done..  will not be used in this notebook)

In [ ]:
documentAssembler = DocumentAssembler()\
      .setInputCol("text")\
      .setOutputCol("ner_chunk")

sbert_embedder = BertSentenceEmbeddings\
      .pretrained('sbiobert_base_cased_mli', 'en','clinical/models')\
      .setInputCols(["ner_chunk"])\
      .setOutputCol("sbert_embeddings")
    
snomed_ct_resolver = SentenceEntityResolverModel.pretrained("sbiobertresolve_snomed_findings","en", "clinical/models") \
      .setInputCols(["ner_chunk", "sbert_embeddings"]) \
      .setOutputCol("snomed_code")\
      .setDistanceFunction("EUCLIDEAN")

icd10_resolver = SentenceEntityResolverModel.pretrained("sbiobertresolve_icd10cm_augmented","en", "clinical/models") \
      .setInputCols(["ner_chunk", "sbert_embeddings"]) \
      .setOutputCol("icd10cm_code")\
      .setDistanceFunction("EUCLIDEAN")

rxnorm_resolver = SentenceEntityResolverModel.pretrained("sbiobertresolve_rxnorm","en", "clinical/models") \
      .setInputCols(["ner_chunk", "sbert_embeddings"]) \
      .setOutputCol("rxnorm_code")\
      .setDistanceFunction("EUCLIDEAN")

cpt_resolver = SentenceEntityResolverModel.pretrained("sbiobertresolve_cpt_procedures_augmented","en", "clinical/models") \
      .setInputCols(["ner_chunk", "sbert_embeddings"]) \
      .setOutputCol("cpt_code")\
      .setDistanceFunction("EUCLIDEAN")

hcc_resolver = SentenceEntityResolverModel.pretrained("sbert_biobertresolve_icd10cm_augmented_billable_hcc","en", "clinical/models") \
      .setInputCols(["ner_chunk", "sbert_embeddings"]) \
      .setOutputCol("icd10cm_hcc_code")\
      .setDistanceFunction("EUCLIDEAN")


resolver_pipelineModel = PipelineModel(
    stages = [
        documentAssembler,
        sbert_embedder,
        snomed_ct_resolver,
        icd10_resolver,
        rxnorm_resolver,
        cpt_resolver,
        hcc_resolver])

resolver_lp = LightPipeline(resolver_pipelineModel)


## Utility functions

In [ ]:
import pandas as pd

pd.set_option('display.max_colwidth', 0)


def get_codes (lp, text, vocab='icd10cm_code', hcc=False):
    
    full_light_result = lp.fullAnnotate(text)

    chunks = []
    codes = []
    begin = []
    end = []
    resolutions=[]
    entities=[]
    all_distances =[]
    all_codes=[]
    all_cosines = []
    all_k_aux_labels=[]

    for chunk, code in zip(full_light_result[0]['ner_chunk'], full_light_result[0][vocab]):
            
        begin.append(chunk.begin)
        end.append(chunk.end)
        chunks.append(chunk.result)
        entities.append(chunk.metadata['entity']) 
        codes.append(code.result) 
        all_codes.append(code.metadata['all_k_results'].split(':::'))
        resolutions.append(code.metadata['all_k_resolutions'].split(':::'))
        all_distances.append(code.metadata['all_k_distances'].split(':::'))
        all_cosines.append(code.metadata['all_k_cosine_distances'].split(':::'))
        if hcc:
            try:
                all_k_aux_labels.append(code.metadata['all_k_aux_labels'].split(':::'))
            except:
                all_k_aux_labels.append([])
        else:
            all_k_aux_labels.append([])

    df = pd.DataFrame({'chunks':chunks, 'begin': begin, 'end':end, 'entity':entities,
                    'code':codes,'all_codes':all_codes, 'resolutions':resolutions, 
                    'all_k_aux_labels':all_k_aux_labels,'all_distances':all_cosines})
    
    if hcc:

        df['billable'] = df['all_k_aux_labels'].apply(lambda x: [i.split('||')[0] for i in x])
        df['hcc_status'] = df['all_k_aux_labels'].apply(lambda x: [i.split('||')[1] for i in x])
        df['hcc_score'] = df['all_k_aux_labels'].apply(lambda x: [i.split('||')[2] for i in x])

    df = df.drop(['all_k_aux_labels'], axis=1)
    
    return df


## Getting some predictions from resolvers

In [ ]:
text = 'bladder cancer'

%time get_codes (icd_lp, text, vocab='icd10cm_code')

CPU times: user 9.58 ms, sys: 6.6 ms, total: 16.2 ms
Wall time: 3.64 s


,chunks,begin,end,entity,code,all_codes,resolutions,all_distances
0,bladder cancer,0,13,None,C679,"[C679, Z126, D090, D494, C7911]","[bladder cancer, suspected bladder cancer, cancer in situ of urinary bladder, tumor of bladder neck, malignant tumour of bladder neck]","[0.0000, 0.0904, 0.0978, 0.1080, 0.1281]"


In [ ]:
text = 'bladder cancer'

%time get_codes (hcc_lp, text, vocab='icd10cm_hcc_code', hcc=True)

CPU times: user 13 ms, sys: 3.88 ms, total: 16.9 ms
Wall time: 706 ms


,chunks,begin,end,entity,code,all_codes,resolutions,all_distances,hcc_status,hcc_score
0,bladder cancer,0,13,None,C679,"[C679, Z126, D090, D494, C7911]","[bladder cancer, suspected bladder cancer, cancer in situ of urinary bladder, tumor of bladder neck, malignant tumour of bladder neck]","[0.0000, 0.0904, 0.0978, 0.1080, 0.1281]","[1, 0, 0, 0, 1]","[11, 0, 0, 0, 8]"


In [ ]:
text = 'severe stomach pain'

%time get_codes (icd_lp, text, vocab='icd10cm_code')

CPU times: user 13.4 ms, sys: 3.02 ms, total: 16.4 ms
Wall time: 706 ms


,chunks,begin,end,entity,code,all_codes,resolutions,all_distances
0,severe stomach pain,0,18,None,R101,"[R101, R109, R1082, M791, R104, R1084, K388, R529, F454, R079, R53, R1013, R52, M7918, R51, R520, R4589, R1033, R072, R103, R1010]","[upper abdominal pain, intractable abdominal pain, epigastric pain, sore pain, generalized colicky abdominal pain, generalized abdominal pain, colicky abdominal pain, severe pain, psychosomatic abdominal pain, upper chest pain, exhausting with pain, burning epigastric pain, heavy pain, abdominal muscle pain, head pain, horrible present pain, cruel with pain, central abdominal pain, esophageal pain, o/e - epigastric pain, ulcer-type pain]","[0.0523, 0.0551, 0.0565, 0.0570, 0.0578, 0.0589, 0.0605, 0.0613, 0.0634, 0.0641, 0.0644, 0.0664, 0.0656, 0.0663, 0.0677, 0.0682, 0.0686, 0.0711, 0.0724, 0.0721, 0.0724]"


In [ ]:
text = 'severe stomach pain'

%time get_codes (hcc_lp, text, vocab='icd10cm_hcc_code', hcc=True)

CPU times: user 18 ms, sys: 0 ns, total: 18 ms
Wall time: 708 ms


,chunks,begin,end,entity,code,all_codes,resolutions,all_distances,hcc_status,hcc_score
0,severe stomach pain,0,18,None,R101,"[R101, R109, R1082, M791, R104, R1084, K388, R529, F454, R079, R53, R1013, R52, M7918, R51, R520, R4589, R1033, R072, R103, R1010]","[upper abdominal pain, intractable abdominal pain, epigastric pain, sore pain, generalized colicky abdominal pain, generalized abdominal pain, colicky abdominal pain, severe pain, psychosomatic abdominal pain, upper chest pain, exhausting with pain, burning epigastric pain, heavy pain, abdominal muscle pain, head pain, horrible present pain, cruel with pain, central abdominal pain, esophageal pain, o/e - epigastric pain, ulcer-type pain]","[0.0523, 0.0551, 0.0565, 0.0570, 0.0578, 0.0589, 0.0605, 0.0613, 0.0634, 0.0641, 0.0644, 0.0664, 0.0656, 0.0663, 0.0677, 0.0682, 0.0686, 0.0711, 0.0724, 0.0721, 0.0724]","[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"


In [ ]:
text = 'bladder cancer'

%time get_codes (snomed_lp, text, vocab='snomed_code')

CPU times: user 8.75 ms, sys: 8.41 ms, total: 17.2 ms
Wall time: 3.65 s


,chunks,begin,end,entity,code,all_codes,resolutions,all_distances
0,bladder cancer,0,13,None,399326009,"[399326009, 363455001, 425066001, 255108000, 269607003, 154540000, 425231005, 255110003, 393562002, 255109008, 255111004, 92546004, 254932004]","[bladder cancer, bladder cancer, invasive bladder cancer, carcinoma of bladder, carcinoma of bladder, carcinoma of bladder, superficial bladder cancer, adenocarcinoma of bladder, transitional cell carcinoma of bladder, transitional cell carcinoma of bladder, squamous cell carcinoma of bladder, cancer in situ of urinary bladder, tumor of bladder neck]","[0.0000, 0.0000, 0.0539, 0.0666, 0.0666, 0.0666, 0.0809, 0.0881, 0.0880, 0.0880, 0.0913, 0.0978, 0.1080]"


In [ ]:
text = '67 yrs-old'

%time get_codes (snomed_lp, text, vocab='snomed_code')

CPU times: user 7.99 ms, sys: 8.04 ms, total: 16 ms
Wall time: 764 ms


,chunks,begin,end,entity,code,all_codes,resolutions,all_distances
0,67 yrs-old,0,9,None,102527003,"[102527003, 102528008, 2391000124102, 111522004, 28288005, 49808004, 260278007, 248280005, 102891000, 152003002, 183774000, 308508003, 249860006, 246944005]","[age 60 to 64 years (finding), age more than 65 years (finding), universal designation 67, gerontoxon, middle-age, geriatric state, 6/60 (finding), aging, age-related cognitive decline (finding), listed for geriatrics admiss'n, listed for geriatrics admiss'n, geriatric monitoring status (finding), posture appropriate for age, pseudo-gerontoxon]","[0.2050, 0.2025, 0.2087, 0.2449, 0.2435, 0.2549, 0.2618, 0.2580, 0.2696, 0.2775, 0.2775, 0.2748, 0.2691, 0.2903]"


In [ ]:
text = 'metformin 100 mg'

%time get_codes (rxnorm_lp, text, vocab='rxnorm_code')

CPU times: user 9.69 ms, sys: 4.89 ms, total: 14.6 ms
Wall time: 486 ms


,chunks,begin,end,entity,code,all_codes,resolutions,all_distances
0,metformin 100 mg,0,15,None,406081,"[406081, 576612, 403968, 861024, 404727, 334738, 332848, 861026, 333262, 439563, 450523, 1744000, 484793, 402346, 1726496, 316350, 858858, 336846, 316844, 1946837, 451225, 328507, 437723, 385601, 315677]","[metformin 100 mg/ml, metformin 100 mg/ml [riomet], metformin 100 mg/ml oral solution, metformin hydrochloride 100 mg/ml, metformin 100 mg/ml oral solution [riomet], fenofibrate 100 mg, ciprofibrate 100 mg, metformin hydrochloride 100 mg/ml [riomet], rutin 100 mg, fendiline 100 mg, perazine 100 mg, emtricitabine 100 mg, solifenacin 100 mg, miglustat 100 mg, azacitidine 100 mg, niacin 100 mg, carnosine 100 mg, trimebutine 100 mg, torsemide 100 mg, abemaciclib 100 mg, pyrantel 100 mg, rimantadine 100 mg, azintamide 100 mg, mebeverine 100 mg, cimetidine 100 mg]","[0.0235, 0.0376, 0.0505, 0.0658, 0.0670, 0.0684, 0.0697, 0.0770, 0.0765, 0.0775, 0.0804, 0.0818, 0.0807, 0.0811, 0.0812, 0.0811, 0.0828, 0.0844, 0.0846, 0.0840, 0.0844, 0.0850, 0.0851, 0.0851, 0.0876]"


In [ ]:
text = 'heart surgery'

%time get_codes (cpt_lp, text, vocab='cpt_code')

CPU times: user 1.52 ms, sys: 12.4 ms, total: 13.9 ms
Wall time: 127 ms


,chunks,begin,end,entity,code,all_codes,resolutions,all_distances
0,heart surgery,0,12,None,33257,"[33257, 33258, 33259, 33999, 0306T, 0304T, 0302T, 0167T, 93583, 1006057, 62319]","[Cardiac surgery procedure, Cardiac surgery procedure, Cardiac surgery procedure, Cardiac surgery procedure, Heart procedure, Heart procedure, Heart procedure, Heart implantation, Operation on cardiac septum, Surgical Procedures on the Heart and Pericardium, Insertion of catheter into heart chamber]","[0.1031, 0.1031, 0.1031, 0.1031, 0.1377, 0.1377, 0.1377, 0.1574, 0.1645, 0.1845, 0.1904]"


In [ ]:
text = 'ct abdomen'

%time get_codes (cpt_lp, text, vocab='cpt_code')

CPU times: user 6.6 ms, sys: 7.78 ms, total: 14.4 ms
Wall time: 130 ms


,chunks,begin,end,entity,code,all_codes,resolutions,all_distances
0,ct abdomen,0,9,None,62284,"[62284, 74183, 74181, 70481, 70480, 76700, 76705, 0157T, 49329, 49999, 61108, 70540, 70543, 71555, 49080, 49083, 33891, 34051, 34001, 35002]","[CT of spine, MRI of abdomen, MRI of abdomen, CT of orbits, CT of orbits, US abdominal scan, US abdominal scan, Opening of abdomen, Procedure on abdomen, Procedure on abdomen, Centesis, MRI of neck, MRI of neck, MRI of chest, Drainage of abdomen, Drainage of abdomen, Incision of neck, Incision of neck, Incision of neck, Incision of neck]","[0.1268, 0.1351, 0.1351, 0.1490, 0.1490, 0.1590, 0.1590, 0.1559, 0.1669, 0.1669, 0.1831, 0.1869, 0.1869, 0.1902, 0.1899, 0.1899, 0.1952, 0.1952, 0.1952, 0.1952]"


In [ ]:
text = 'ct abdomen'

%time get_codes (cpt_lp, text, vocab='cpt_code')

CPU times: user 6.6 ms, sys: 7.78 ms, total: 14.4 ms
Wall time: 130 ms


,chunks,begin,end,entity,code,all_codes,resolutions,all_distances
0,ct abdomen,0,9,None,62284,"[62284, 74183, 74181, 70481, 70480, 76700, 76705, 0157T, 49329, 49999, 61108, 70540, 70543, 71555, 49080, 49083, 33891, 34051, 34001, 35002]","[CT of spine, MRI of abdomen, MRI of abdomen, CT of orbits, CT of orbits, US abdominal scan, US abdominal scan, Opening of abdomen, Procedure on abdomen, Procedure on abdomen, Centesis, MRI of neck, MRI of neck, MRI of chest, Drainage of abdomen, Drainage of abdomen, Incision of neck, Incision of neck, Incision of neck, Incision of neck]","[0.1268, 0.1351, 0.1351, 0.1490, 0.1490, 0.1590, 0.1590, 0.1559, 0.1669, 0.1669, 0.1831, 0.1869, 0.1869, 0.1902, 0.1899, 0.1899, 0.1952, 0.1952, 0.1952, 0.1952]"


In [ ]:
icd_chunks = ['advanced liver disease',
'advanced lung disease',
'basal cell carcinoma of skin',
'acute maxillary sinusitis',
'chronic kidney disease stage',
'diabetes mellitus type 2',
'lymph nodes of multiple sites',
'other chronic pain',
'severe abdominal pain',
'squamous cell carcinoma of skin',
'type 2 diabetes mellitus']

snomed_chunks= ['anemia', 'adenocarcinoma', 'aortic valve stenosis',
       'atherosclerosis', 'atrial fibrillation', 'dyspnea',
       'hypertension', 'lung cancer', 'seizure',
       'squamous cell carcinoma', 'stage IIIB', 'mediastinal lymph nodes']

In [ ]:
from IPython.display import display

for chunk in icd_chunks:

    print ('>> ',chunk)
    display(get_codes (icd_lp, chunk, vocab='icd10cm_code'))

>>  advanced liver disease


,chunks,begin,end,entity,code,all_codes,resolutions,all_distances
0,advanced liver disease,0,21,None,K7460,"[K7460, K769, K721, K746, R945, Z871, K7402, K729, K7290, K7689, D489, D499, R748, K719]","[advanced cirrhosis, chronic liver disease, chronic hepatic failure, cirrhosis and chronic liver disease, abnormal liver function, h/o: liver disease, hepatic fibrosis, advanced fibrosis, liver function failure, liver failure, abnormal hepatic function, tumor of advanced extent, tumour of advanced extent, elevated liver enzymes level, toxic liver disease]","[0.0704, 0.1057, 0.1081, 0.1263, 0.1299, 0.1324, 0.1338, 0.1319, 0.1334, 0.1425, 0.1468, 0.1477, 0.1487, 0.1517]"


>>  advanced lung disease


,chunks,begin,end,entity,code,all_codes,resolutions,all_distances
0,advanced lung disease,0,20,None,D499,"[D499, D489, J449, H359, J849, J628, K7460, J399, I5083, J961, J989, I509, J8410, R54, J984, I279]","[tumour of advanced extent, tumor of advanced extent, severe chronic obstructive pulmonary disease, advanced retinal disease, chronic infiltrative lung disease, chronic lung disease, advanced cirrhosis, upper respiratory disease, high output heart failure, chronic respiratory failure, chronic respiratory system disease, cardiac failure, progressive lung fibrosis, advanced age, chronic lung disease (disorder), chronic pulmonary heart disease]","[0.1284, 0.1333, 0.1329, 0.1353, 0.1355, 0.1373, 0.1455, 0.1471, 0.1468, 0.1537, 0.1565, 0.1572, 0.1588, 0.1597, 0.1641, 0.1644]"


>>  basal cell carcinoma of skin


...

,chunks,begin,end,entity,code,all_codes,resolutions,all_distances
0,severe abdominal pain,0,20,None,R1084,"[R1084, R101, R109, K388, R104, R1033, R102, F454, K805, R529, R52, M2551]","[generalized abdominal pain, upper abdominal pain, intractable abdominal pain, colicky abdominal pain, generalized colicky abdominal pain, central abdominal pain, acute abdominal pain, psychosomatic abdominal pain, recurrent abdominal pain, severe pain, heavy pain, shoulder pain from abdomen]","[0.0196, 0.0208, 0.0208, 0.0318, 0.0322, 0.0352, 0.0370, 0.0370, 0.0408, 0.0424, 0.0489, 0.0507]"


>>  squamous cell carcinoma of skin


,chunks,begin,end,entity,code,all_codes,resolutions,all_distances
0,squamous cell carcinoma of skin,0,30,None,C449,"[C449, C443, D049, C4492, C4452, C4499, C4432, C809, C4442, C4A70, C444, C4491, C799]","[squamous cell carcinoma of skin, squamous cell carcinoma of skin of face, squamous cell carcinoma in situ of skin, scc - squamous cell carcinoma of skin, squamous cell carcinoma of skin of trunk, mucoepidermoid carcinoma of skin, squamous cell carcinoma of skin of and unsp parts of face, squamous cell carcinoma, squamous cell carcinoma of skin of neck, trabecular cell carcinoma of skin, squamous cell carcinoma of scalp, basal cell carcinoma of skin, squamous cell carcinomatosis]","[0.0000, 0.0244, 0.0289, 0.0333, 0.0337, 0.0420, 0.0519, 0.0547, 0.0564, 0.0600, 0.0623, 0.0620, 0.0638]"


>>  type 2 diabetes mellitus


,chunks,begin,end,entity,code,all_codes,resolutions,all_distances
0,type 2 diabetes mellitus,0,23,None,E119,"[E119, E118, E113, O2411, E139, E117, E1143, E109, E1169, E114, E112, E1151, E1162]","[type 2 diabetes mellitus, disorder associated with type 2 diabetes mellitus, disorder of eye with type 2 diabetes mellitus, pre-existing type 2 diabetes mellitus, secondary diabetes mellitus, multiple complications of type 2 diabetes mellitus, neurological disorder with type 2 diabetes mellitus, diabetes mellitus, diabetic dyslipidemia associated with type 2 diabetes mellitus, neurological disorder with diabetes type 2, kidney disorder associated with type 2 diabetes mellitus, peripheral circulatory disorder associated with type 2 diabetes mellitus, type 2 diabetes mellitus with skin complications]","[0.0000, 0.0212, 0.0357, 0.0360, 0.0397, 0.0410, 0.0455, 0.0462, 0.0499, 0.0523, 0.0537, 0.0548, 0.0549]"


In [ ]:
for chunk in snomed_chunks:

    print ('>> ',chunk)
    display(get_codes (snomed_lp, chunk, vocab='snomed_code'))

>>  anemia


,chunks,begin,end,entity,code,all_codes,resolutions,all_distances
0,anemia,0,5,None,271737000,"[271737000, 64593003, 154786001, 11503009, 191268006, 35372003, 191125001, 267513007, 234347009, 737220002, 234360003, 141336002, 164139008, 84027009, 191139001, 154789008]","[anemia, anemia, anemia, relative anemia, chronic anemia, aids with anemia, deficiency anemias, deficiency anemias, secondary anemia, metabolic anemia, vegan anemia, o/e - anemia, o/e - anemia, pernicious anemia, pernicious anemia, pernicious anemia]","[0.0000, 0.0000, 0.0000, 0.0435, 0.0439, 0.0475, 0.0503, 0.0503, 0.0658, 0.0682, 0.0680, 0.0690, 0.0690, 0.0751, 0.0751, 0.0751]"


>>  adenocarcinoma


,chunks,begin,end,entity,code,all_codes,resolutions,all_distances
0,adenocarcinoma,0,13,None,443961001,"[443961001, 403902008, 423595004, 189578007, 189614008, 423607006, 722688002, 154433003, 269513004, 269546005, 443416007, 403951006, 307599002, 441535001, 307593001, 154576004, 269624009, 188476000, 443261008]","[adenocarcinoma, adenosquamous carcinoma, adenocarcinoma carcinomatosis, [m]adenocarcinomas, adenocarcinoid tumor, adenocarcinoma of anus, carcinoma, carcinoma, carcinoma, anus carcinoma, adenoma, ceruminous gland adenocarcinoma, sebaceous adenocarcinoma, adenocarcinoma of head and neck, carcinomatosis, carcinomatosis, carcinomatosis, carcinomatosis, oxyphilic adenocarcinoma]","[0.0000, 0.0401, 0.0546, 0.0586, 0.0615, 0.0774, 0.0790, 0.0790, 0.0790, 0.0796, 0.0847, 0.0884, 0.0903, 0.0897, 0.0948, 0.0948, 0.0948, 0.0948, 0.0960]"


>>  aortic valve stenosis


...

,chunks,begin,end,entity,code,all_codes,resolutions,all_distances
0,squamous cell carcinoma,0,22,None,402815007,"[402815007, 154605007, 425303004, 254651007, 403904009, 189565007, 403905005, 403902008, 400066006, 403903003, 189568009, 403899005, 254656002, 254657006, 423284006, 403892001, 716659002]","[squamous cell carcinoma, squamous cell carcinoma, squamous cell carcinomatosis, cutaneous squamous cell carcinoma, verrucous squamous cell carcinoma, squamous cell carcinoma in situ, multiple squamous cell carcinomata, adenosquamous cell carcinoma, intraepithelial squamous cell carcinoma, signet ring squamous cell carcinoma, keratinising squamous cell carcinoma, squamous cell carcinoma of skin of trunk, squamous cell carcinoma in situ of skin, squamous cell carcinoma in situ of skin, squamous cell carcinoma of skin of neck, squamous cell carcinoma of skin of face, squamous cell carcinoma of head and neck]","[0.0000, 0.0000, 0.0191, 0.0398, 0.0455, 0.0468, 0.0495, 0.0498, 0.0566, 0.0621, 0.0628, 0.0679, 0.0672, 0.0672, 0.0695, 0.0730, 0.0736]"


>>  stage IIIB


,chunks,begin,end,entity,code,all_codes,resolutions,all_distances
0,stage IIIB,0,9,None,64062008,"[64062008, 73082003, 45597001, 443815000, 50283003, 48105005, 64877001, 443962008, 55336000, 33907000, 433144002, 443817008, 61026006, 52708004, 434002]","[clinical stage 3 b, clinical stage 3 a, group a3b, pt3a,b category, clinical stage iii, clinical stage 3 c, clinical stage iii s, pn3b category, clinical stage iii d, clinical stage iii e, ckd stage 3, pt3b category, g3 stage, figo cc stage iii, ebr 3]","[0.0771, 0.1143, 0.1326, 0.1356, 0.1327, 0.1361, 0.1451, 0.1540, 0.1472, 0.1490, 0.1592, 0.1659, 0.1885, 0.1908, 0.1929]"


>>  mediastinal lymph nodes


,chunks,begin,end,entity,code,all_codes,resolutions,all_distances
0,mediastinal lymph nodes,0,22,None,301296002,"[301296002, 127248001, 49483002, 94147001, 234317007, 52324001, 127245003, 30746006, 274711004, 19616004, 126725000, 301855007, 425061006, 127274007, 76616003, 127232002, 127235000, 127234001]","[mediastinal observation, neoplasm of mediastinal lymph nodes, mediastinal disease, mediastinal mass, lymphangiography of mediastinal lymph nodes, mediastinal lymphadenopathy, neoplasm of intrathoracic lymph nodes, swelling of lymph nodes, mediastinal shift, mediastinal shift, tumour of mediastinum, finding of lymph node, lymphadenopathy of head and/or neck, neoplasm of lymph nodes of multiple sites, disease of lymph node, neoplasm of lymph node, neoplasm of lymph nodes of neck, neoplasm of lymph nodes of face]","[0.0849, 0.0879, 0.0954, 0.1123, 0.1319, 0.1254, 0.1351, 0.1325, 0.1327, 0.1327, 0.1434, 0.1392, 0.1380, 0.1426, 0.1439, 0.1473, 0.1481, 0.1503]"


In [ ]:
clinical_chunks = ['bladder cancer',
 'anemia in chronic kidney disease',
 'castleman disease',
 'congestive heart failure',
 'diabetes mellitus type 2',
 'lymph nodes of multiple sites',
 'malignant melanoma of skin',
 'malignant neoplasm of lower lobe, bronchus',
 'metastatic lung cancer',
 'secondary malignant neoplasm of bone',
 'type 2 diabetes mellitus',
 'type 2 diabetes mellitus/insulin',
 'unsp malignant neoplasm of lymph node']


for chunk in clinical_chunks:

    print ('>> ',chunk)
    
    print ('icd10cm_code')
    display(get_codes (icd_lp, chunk, vocab='icd10cm_code'))
    
    print ('snomed_code')
    display(get_codes (snomed_lp, chunk, vocab='snomed_code'))

>>  bladder cancer
icd10cm_code


,chunks,begin,end,entity,code,all_codes,resolutions,all_distances
0,bladder cancer,0,13,None,C679,"[C679, Z126, D090, D494, C7911]","[bladder cancer, suspected bladder cancer, cancer in situ of urinary bladder, tumor of bladder neck, malignant tumour of bladder neck]","[0.0000, 0.0904, 0.0978, 0.1080, 0.1281]"


snomed_code


,chunks,begin,end,entity,code,all_codes,resolutions,all_distances
0,bladder cancer,0,13,None,399326009,"[399326009, 363455001, 425066001, 255108000, 269607003, 154540000, 425231005, 255110003, 393562002, 255109008, 255111004, 92546004, 254932004]","[bladder cancer, bladder cancer, invasive bladder cancer, carcinoma of bladder, carcinoma of bladder, carcinoma of bladder, superficial bladder cancer, adenocarcinoma of bladder, transitional cell carcinoma of bladder, transitional cell carcinoma of bladder, squamous cell carcinoma of bladder, cancer in situ of urinary bladder, tumor of bladder neck]","[0.0000, 0.0000, 0.0539, 0.0666, 0.0666, 0.0666, 0.0809, 0.0881, 0.0880, 0.0880, 0.0913, 0.0978, 0.1080]"


>>  anemia in chronic kidney disease
icd10cm_code


...

,chunks,begin,end,entity,code,all_codes,resolutions,all_distances
0,type 2 diabetes mellitus/insulin,0,31,None,237599002,"[237599002, 366909003, 44054006, 368051000119109, 422014003, 422099009, 199230006, 137931000119102, 190384004, 421326000]","[insulin treated type 2 diabetes mellitus, insulin treated type 2 diabetes mellitus, type 2 diabetes mellitus, hyperglycemia due to type 2 diabetes mellitus (disorder), disorder associated with type 2 diabetes mellitus, disorder of eye with type 2 diabetes mellitus, pre-existing type 2 diabetes mellitus (disorder), hyperlipidaemia due to type 2 diabetes mellitus, type ii diabetes mellitus, disorder of nervous system due to type 2 diabetes mellitus]","[0.0513, 0.0513, 0.0554, 0.0593, 0.0606, 0.0658, 0.0686, 0.0690, 0.0708, 0.0718]"


>>  unsp malignant neoplasm of lymph node
icd10cm_code


,chunks,begin,end,entity,code,all_codes,resolutions,all_distances
0,unsp malignant neoplasm of lymph node,0,36,None,C779,"[C779, C77, D360, C772, C775, D492, D219]","[secondary and unsp malignant neoplasm of lymph node, unsp, secondary and unspecified malignant neoplasm of lymph nodes, benign neoplasm of lymph node, secondary and unsp malignant neoplasm of intra-abd nodes, secondary and unsp malignant neoplasm of intrapelv nodes, neoplasm of lymph node, benign neoplasm of lymph vessels]","[0.0606, 0.0681, 0.0754, 0.0920, 0.1034, 0.0968, 0.1049]"


snomed_code


,chunks,begin,end,entity,code,all_codes,resolutions,all_distances
0,unsp malignant neoplasm of lymph node,0,36,None,188379006,"[188379006, 271526003, 188432007, 92197001, 189203003, 188706002, 92202008, 92156008, 188707006, 188713002, 92090002, 92203003, 188392009, 188393004, 188709009, 188389005, 188715009, 92199003, 92213006, 127232002, 188423005, 92201001]","[secondary and unspecified malignant neoplasm of lymph nodes, secondary and unspecified malignant neoplasm of lymph nodes, secondary and unspecified malignant neoplasm of lymph nodes nos, benign neoplasm of lymph node, benign neoplasm of lymph nodes, unspecified malignant neoplasm of lymphoid and histiocytic tissue of unspecified site, benign neoplasm of lymph nodes of neck, benign neoplasm of intrathoracic lymph nodes, unspecified malignant neoplasm of lymphoid and histiocytic tissue of lymph nodes of head, face and neck, unspecified malignant neoplasm of lymphoid and histiocytic tissue of intrapelvic lymph nodes, benign neoplasm of epitrochlear lymph nodes, benign neoplasm of lymph nodes of upper limb, secondary and unspecified malignant neoplasm of lymph nodes of head, face and neck nos, secondary and unspecified malignant neoplasm of intrathoracic lymph nodes, unspecified malignant neoplasm of lymphoid and histiocytic tissue of intrathoracic lymph nodes, secondary and unspecified malignant neoplasm of submental lymph nodes, unspecified malignant neoplasm of lymphoid and histiocytic tissue of lymph nodes of multiple sites, benign neoplasm of lymph nodes of head (disorder), benign neoplasm of mediastinal lymph nodes, neoplasm of lymph node, secondary and unspecified malignant neoplasm of intrapelvic lymph nodes, benign neoplasm of lymph nodes of multiple sites (disorder)]","[0.0681, 0.0681, 0.0739, 0.0754, 0.0758, 0.0841, 0.0860, 0.0896, 0.0948, 0.0963, 0.0954, 0.0938, 0.0969, 0.0980, 0.0995, 0.0994, 0.1006, 0.0971, 0.0985, 0.0968, 0.1032, 0.1004]"


##  How to integrate resolvers with NER models in the same pipeline


In [ ]:
documentAssembler = DocumentAssembler()\
      .setInputCol("text")\
      .setOutputCol("document")

sentenceDetector = SentenceDetectorDLModel.pretrained()\
      .setInputCols(["document"])\
      .setOutputCol("sentence")

tokenizer = Tokenizer()\
      .setInputCols(["sentence"])\
      .setOutputCol("token")\

word_embeddings = WordEmbeddingsModel.pretrained("embeddings_clinical", "en", "clinical/models")\
      .setInputCols(["sentence", "token"])\
      .setOutputCol("embeddings")

clinical_ner = MedicalNerModel.pretrained("ner_clinical", "en", "clinical/models") \
      .setInputCols(["sentence", "token", "embeddings"]) \
      .setOutputCol("ner")

ner_converter = NerConverter() \
      .setInputCols(["sentence", "token", "ner"]) \
      .setOutputCol("ner_chunk")\
      .setWhiteList(['PROBLEM'])

c2doc = Chunk2Doc()\
      .setInputCols("ner_chunk")\
      .setOutputCol("ner_chunk_doc") 

sbert_embedder = BertSentenceEmbeddings\
      .pretrained("sbiobert_base_cased_mli",'en','clinical/models')\
      .setInputCols(["ner_chunk_doc"])\
      .setOutputCol("sbert_embeddings")

icd10_resolver = SentenceEntityResolverModel.pretrained("sbiobertresolve_icd10cm_augmented","en", "clinical/models") \
      .setInputCols(["ner_chunk", "sbert_embeddings"]) \
      .setOutputCol("icd10cm_code")\
      .setDistanceFunction("EUCLIDEAN")
  
sbert_resolver_pipeline = Pipeline(
    stages = [
        documentAssembler,
        sentenceDetector,
        tokenizer,
        word_embeddings,
        clinical_ner,
        ner_converter,
        c2doc,
        sbert_embedder,
        icd10_resolver])

data_ner = spark.createDataFrame([[""]]).toDF("text")

sbert_models = sbert_resolver_pipeline.fit(data_ner)



sentence_detector_dl download started this may take some time.
Approximate size to download 354.6 KB
[OK!]
embeddings_clinical download started this may take some time.
Approximate size to download 1.6 GB
[OK!]
ner_clinical download started this may take some time.
Approximate size to download 13.9 MB
[OK!]
sbiobert_base_cased_mli download started this may take some time.
Approximate size to download 384.3 MB
[OK!]
sbiobertresolve_icd10cm_augmented download started this may take some time.
Approximate size to download 1.2 GB
[OK!]


In [ ]:
clinical_note = 'A 28-year-old female with a history of gestational diabetes mellitus diagnosed eight years prior to presentation and subsequent type two diabetes mellitus (T2DM), one prior episode of HTG-induced pancreatitis three years prior to presentation, associated with an acute hepatitis, and obesity with a body mass index (BMI) of 33.5 kg/m2, presented with a one-week history of polyuria, polydipsia, poor appetite, and vomiting. Two weeks prior to presentation, she was treated with a five-day course of amoxicillin for a respiratory tract infection. She was on metformin, glipizide, and dapagliflozin for T2DM and atorvastatin and gemfibrozil for HTG. She had been on dapagliflozin for six months at the time of presentation. Physical examination on presentation was significant for dry oral mucosa; significantly, her abdominal examination was benign with no tenderness, guarding, or rigidity. Pertinent laboratory findings on admission were: serum glucose 111 mg/dl, bicarbonate 18 mmol/l, anion gap 20, creatinine 0.4 mg/dL, triglycerides 508 mg/dL, total cholesterol 122 mg/dL, glycated hemoglobin (HbA1c) 10%, and venous pH 7.27. Serum lipase was normal at 43 U/L. Serum acetone levels could not be assessed as blood samples kept hemolyzing due to significant lipemia. The patient was initially admitted for starvation ketosis, as she reported poor oral intake for three days prior to admission. However, serum chemistry obtained six hours after presentation revealed her glucose was 186 mg/dL, the anion gap was still elevated at 21, serum bicarbonate was 16 mmol/L, triglyceride level peaked at 2050 mg/dL, and lipase was 52 U/L. The β-hydroxybutyrate level was obtained and found to be elevated at 5.29 mmol/L - the original sample was centrifuged and the chylomicron layer removed prior to analysis due to interference from turbidity caused by lipemia again. The patient was treated with an insulin drip for euDKA and HTG with a reduction in the anion gap to 13 and triglycerides to 1400 mg/dL, within 24 hours. Her euDKA was thought to be precipitated by her respiratory tract infection in the setting of SGLT2 inhibitor use. The patient was seen by the endocrinology service and she was discharged on 40 units of insulin glargine at night, 12 units of insulin lispro with meals, and metformin 1000 mg two times a day. It was determined that all SGLT2 inhibitors should be discontinued indefinitely. She had close follow-up with endocrinology post discharge.'

print (clinical_note)

clinical_note_df = spark.createDataFrame([[clinical_note]]).toDF("text")


A 28-year-old female with a history of gestational diabetes mellitus diagnosed eight years prior to presentation and subsequent type two diabetes mellitus (T2DM), one prior episode of HTG-induced pancreatitis three years prior to presentation, associated with an acute hepatitis, and obesity with a body mass index (BMI) of 33.5 kg/m2, presented with a one-week history of polyuria, polydipsia, poor appetite, and vomiting. Two weeks prior to presentation, she was treated with a five-day course of amoxicillin for a respiratory tract infection. She was on metformin, glipizide, and dapagliflozin for T2DM and atorvastatin and gemfibrozil for HTG. She had been on dapagliflozin for six months at the time of presentation. Physical examination on presentation was significant for dry oral mucosa; significantly, her abdominal examination was benign with no tenderness, guarding, or rigidity. Pertinent laboratory findings on admission were: serum glucose 111 mg/dl, bicarbonate 18 mmol/l, anion gap 20

In [ ]:
from pyspark.sql import functions as F

icd10_sdf = sbert_models.transform(clinical_note_df)

icd10_sdf = icd10_sdf.select(F.explode(F.arrays_zip("ner_chunk.result","ner_chunk.metadata","icd10cm_code.result","icd10cm_code.metadata","ner_chunk.begin","ner_chunk.end")).alias("icd10cm_code")) \
    .select(F.expr("icd10cm_code['0']").alias("chunk"),
            F.expr("icd10cm_code['4']").alias("begin"),
            F.expr("icd10cm_code['5']").alias("end"),
            F.expr("icd10cm_code['1'].entity").alias("entity"),
            F.expr("icd10cm_code['2']").alias("code"),
            F.expr("icd10cm_code['3'].confidence").alias("confidence"),
            F.expr("icd10cm_code['3'].all_k_resolutions").alias("all_k_resolutions"),
            F.expr("icd10cm_code['3'].all_k_results").alias("all_k_codes"))

icd10_sdf.show()


+--------------------+-----+----+-------+-----+----------+--------------------+--------------------+
|               chunk|begin| end| entity| code|confidence|   all_k_resolutions|         all_k_codes|
+--------------------+-----+----+-------+-----+----------+--------------------+--------------------+
|gestational diabe...|   39|  67|PROBLEM|O2441|    0.9777|gestational diabe...|O2441:::O2443:::Z...|
|subsequent type t...|  117| 153|PROBLEM|O2411|    0.1711|pre-existing type...|O2411:::E118:::E1...|
|                T2DM|  156| 159|PROBLEM| E119|    0.2436|type 2 diabetes m...|E119:::E118:::O24...|
|HTG-induced pancr...|  184| 207|PROBLEM|K8520|    0.2189|alcohol-induced p...|K8520:::K853:::K8...|
|  an acute hepatitis|  260| 277|PROBLEM| B159|    0.3662|acute hepatitis a...|B159:::K720:::B17...|
|             obesity|  284| 290|PROBLEM| E669|    0.9193|obesity:::abdomin...|E669:::E668:::Z68...|
|   a body mass index|  297| 313|PROBLEM|Z6841|    0.6263|finding of body m...|Z6841:::E669

In [ ]:
# with light pipeline

light_model = LightPipeline(sbert_models)

In [ ]:
from sparknlp_display import EntityResolverVisualizer

vis = EntityResolverVisualizer()

# Change color of an entity label
vis.set_label_colors({'PROBLEM':'#008080'})

light_data_icd = light_model.fullAnnotate(clinical_note)

vis.display(light_data_icd[0], 'ner_chunk', 'icd10cm_code')
